Our project looks to examine the key factors influencing NBA player scoring, utilizing a comprehensive dataset of player statistics from 1950 to 2017. By examining a wide range of features from our dataset—including player position, minutes played, field goal percentage, and usage rate—we seek to identify the elements that contribute most significantly to a player's scoring ability, and to be able to predict future stats with our supervised learning model (supervised because we have quantitative labeled data). In addition, we want to know which combinations of these features are the best predictors for high point totals in an NBA player's entire career. The insights gained from this study can be a valuable resource for sports bettors, fantasy basketball players, and even the coaches or GMs who manage the team themselves when trying to pick players with the best scoring potential.

In [ ]:
!git clone https://github.com/jonathanaduong/CSE151AGroupProject.git

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

In [ ]:
nbadf = pd.read_csv('CSE151AGroupProject/Seasons_Stats.csv')
nbadf.head()

In [ ]:
nbadf.dtypes

In [ ]:
print(nbadf.Pos.unique())

In [ ]:
nbadf.describe()

In [ ]:
print(nbadf.Tm.unique())

In [ ]:
team_counts = nbadf['Tm'].value_counts()

plt.figure(figsize=(12, 6))
team_counts.plot(kind='bar', color='skyblue')
plt.title('Frequency of People in Each Team')
plt.xlabel('Team')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

In [ ]:
sns.pairplot(nbadf[['PTS', 'G', 'MP', 'FG%', 'FGA', 'FT%', 'FTA']])

In [ ]:
corr = nbadf[['PTS', 'G', 'MP', 'FG%', 'FGA', 'FT%', 'FTA']].corr()
sns.heatmap(corr, vmin=-1, vmax=1, center=0, annot=True, cmap= 'RdBu')

In [ ]:
nbadf.isnull().sum()

The data set tracks players' statistics season to season from 1950-2017. The categorical features we plan on observing are Pos (Player position) and Tm (Team they played for) and the numerical features that we plan to observe are G (Games played), MP (Total minutes played), FG% (Percentage hots made in a season), FGA (Shots attempted in a season), FT% (Percentage of free throws made in a season), and FTA (Free throws attempted in a season).

We plan on predicting the points scored per game for an individual, so we need to observe the games played each season for a player (G) as FGA, FTA, and MP are the totals for points scored, field goals shot, free throws shot, and minutes played for the season rather than a per game average, which we would prefer to analyze. We will also be seeing if player position makes a difference in the point scored per game.

The data set has 24,691 different observations, although there are repeat players since they played across multiple years. For scale, we plan to use minMax scaling to ensure that the data is normalized. There are many null/NaN values across seasons since these statistics were not recorded at the time, such as blocks or steals.


### How will you preprocess your data :
- to (one-hot) encode categorical data like Team, Positions
- for null/nan values, we will replace it with an adjusted mean values
- for each of the season total categories (PTS, TRB, AST), we will divide it by the games played to get the per game average.
- use min-max to scale since we do

In [ ]:
processed_nbadf = nbadf.copy()
removeFeat =  ['Unnamed: 0', 'blanl' , 'blank2', 'GS', 'Player', 'ORB', 'DRB', '3P', '2P', 'FG']
processed_nbadf = processed_nbadf.drop(removeFeat, axis=1)

#fill NaNs in each non-category column
for i in processed_nbadf.columns:
  if i == 'Year': continue
  if i == 'MP': continue
  if processed_nbadf[i].dtype == 'object': continue
  #by mean imputation grouped by year:
  processed_nbadf[i] = processed_nbadf.groupby('Year')[i].transform(lambda x: x.fillna(x.mean()))
  col_mean = processed_nbadf[i].mean()
  #by global average:
  processed_nbadf[i] = processed_nbadf[i].fillna(col_mean)

#compute average MP per game
avg_MP = processed_nbadf["MP"].mean()
avg_G = processed_nbadf["G"].mean()
processed_nbadf['MP_per_game'] = processed_nbadf['MP'] / processed_nbadf['G']
#fill NaNs with the global average
processed_nbadf['MP_per_game'] = processed_nbadf['MP_per_game'].fillna(avg_MP / avg_G)

#remove remaining NaNs
processed_nbadf.dropna(inplace= True)

# #one-hot encode categories
processed_nbadf = pd.get_dummies(processed_nbadf, columns=['Tm', 'Pos'])
processed_nbadf

#compute per game and fill out na values with mean
feature_cols = []
for column in ['PTS', 'TRB', 'AST', 'MP', 'FGA', '3PA', '2PA', 'FTA']:
  col = column + '_per_game'
  feature_cols.append(col)
  processed_nbadf[col] = processed_nbadf[column] / processed_nbadf['G']
  processed_nbadf.drop(column, axis = 1)

processed_nbadf.head()

## Model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(processed_nbadf[['TRB_per_game', 'AST_per_game', 'MP_per_game', 'FGA_per_game', '3PA_per_game', '2PA_per_game', 'FTA_per_game']], processed_nbadf.PTS_per_game, test_size=0.2, random_state=21)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

In [ ]:
print(X_train.shape[0]/processed_nbadf.shape[0])
print(X_test.shape[0]/processed_nbadf.shape[0])

In [ ]:
for i in ['TRB_per_game', 'AST_per_game', 'MP_per_game', 'FGA_per_game', '3PA_per_game', '2PA_per_game', 'FTA_per_game']:
  linearreg = LinearRegression()
  linearmodel = linearreg.fit(X_train[i].values.reshape(-1,1), y_train)
  yhat_test = linearreg.predict(X_test[i].values.reshape(-1,1))
  yhat_train = linearreg.predict(X_train[i].values.reshape(-1,1))
  print('Testing MSE: %.2f' % mean_squared_error(y_test, yhat_test))
  plt.scatter(X_train[i],y_train, s=10)
  plt.xlabel(i)
  plt.ylabel('PTS_per_game')
  plt.show()

In [ ]:
  linearreg = LinearRegression()
  ## y_train and y_test serve as ground truth labels, representing the actual points per game derived from our dataset. These values were calculated by dividing each player’s total points in the season by the number of games played, providing a reliable basis for evaluating the model's predictions.
  linearmodel = linearreg.fit(X_train, y_train)
  print(linearmodel.coef_)
  yhat_test = linearreg.predict(X_test)
  yhat_train = linearreg.predict(X_train)
  print('Testing MSE: %.2f' % mean_squared_error(y_test, yhat_test))
  print('Training MSE: %.2f' % mean_squared_error(y_train, yhat_train))

## Conclusion
The conclusion of our 1st model shows our model can achieve a moderate level of accuracy in predicting PPG. To improve our model in the future, we might want to exclude most of the data from 1950s to 1980s because lots of the data had to be imputed and are carried by global averages.

(dionne's attempt code)
- instead of hyperparameter tuning maybe we can try GD?

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(processed_nbadf[['TRB_per_game', 'AST_per_game', 'MP_per_game', 'FGA_per_game', '3PA_per_game', '2PA_per_game', 'FTA_per_game','WS','BPM', 'PER']], processed_nbadf.PTS_per_game, test_size=0.2, random_state=21)

# min max scale
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# turn X_train and X_test into
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

## for the fitting graph
train_mse_list = []
test_mse_list = []

# polynomial degree to add complexity to the model
degrees = [1,2,3,4]
for degree in degrees:
  initialDegree = degree
  poly = PolynomialFeatures(degree=initialDegree)
  X_train_poly = poly.fit_transform(X_train)
  X_test_poly = poly.transform(X_test)

  model2 = LinearRegression()
  model2.fit(X_train_poly, y_train)
  y_train_pred = model2.predict(X_train_poly)
  y_test_pred = model2.predict(X_test_poly)


  train_mse = mean_squared_error(y_train, y_train_pred)
  test_mse = mean_squared_error(y_test, y_test_pred)
  train_mse_list.append(train_mse)
  test_mse_list.append(test_mse)

  print(f"Polynomial Regression (Degree={initialDegree})")
  print(f"Train MSE: {train_mse:.2f}")
  print(f"Test MSE: {test_mse:.2f}")

## WILL CHOOSE Degree 3 for Polynomial Regression, better performance with
## least signs of overfitting
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(degrees, train_mse_list, marker='o', label='Train MSE')
plt.plot(degrees, test_mse_list, marker='o', label='Test MSE')
plt.xlabel('Polynomial Degree')
plt.ylabel('Mean Squared Error')
plt.title('Model Fitting: Train vs Test MSE')
plt.legend()
plt.grid(True)
plt.show()

3. Our model lies in the middle of this fitting graph, where degree equals 3. A decently complex model, which makes sure we are not underfitting while also minimizing the difference between our training and testing error

In [ ]:
poly = PolynomialFeatures(degree=3)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
model2 = LinearRegression()
model2.fit(X_train_poly, y_train)
y_test_pred = model2.predict(X_test_poly)
y_test_pred
threshold = 1
newY = y_test - y_test_pred

## in the context of this question any Correct is ppg predicted within .5 of the true values
## while FN and FP are calculated as any values not within .5 of the true values
correct = len(newY[abs(newY) < .5])
FNFP = len(y_test) - correct
print(f'Correct Values: ', correct)
print(f'FNFP Values: ', FNFP)

#Model 3
For our third model, we will predict a player's **Points Per Game** as a **Category**. Specifically, we will bin PPG as follows:

|Bin|Score|
|----------|-------|
|PPG <= 25%|"Low"|
|25% < PPG <= 50%|"Medium"|
|50% < PPG <= 75%|"High"|
|PPG > 75%|"Very High"|

The thresholds are printed below:

In [ ]:
cols = ['PTS_per_game']
#ppg mean is 8.3; max is 50
desc = processed_nbadf[cols].describe()
desc

We will perform additional preprocessing for this version of our data by adding a new column into our modified DataFrame called "PPG Bin"

In [ ]:
bins = {"Low": desc["PTS_per_game"]["25%"], "Medium": desc["PTS_per_game"]["50%"], "High": desc["PTS_per_game"]["75%"]}
print(bins)
nbadf_2 = processed_nbadf.copy()

#Convert keys into numeric vals, Very High = 3
def PPG_binner(value):
    if value < bins['Low']:
        return 0
    elif value < bins['Medium']:
        return 1
    elif value < bins['High']:
        return 2
    else:
        return 3

#Apply the function to PTS_per_game
nbadf_2['PPG_Bins'] = nbadf_2['PTS_per_game'].apply(PPG_binner)
nbadf_2 = nbadf_2.drop("PTS_per_game", axis=1)
#Drop the PTS_per_game column
nbadf_2.head()

Now we can perform either PCA or SVD using the same dimensions as we did in our previous model. 

We will not take into account the position of a player. This is because PCA using positions results in 5 clusters, binned most likely by the 5 main positions in a basketball team. Within those clusters, observations are clustered by PPG bins. A graph of the results is seen below.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    nbadf_2.iloc[:, -7:-1], 
    nbadf_2.PPG_Bins, 
    test_size=0.2, 
    random_state=21)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
X_train.head()

In [ ]:
y_train.head()

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_train)

ev = pca.explained_variance_ratio_

# Scree plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(ev) + 1), ev, marker='o', linestyle='--')
plt.title('Scree Plot')
plt.xlabel('n_components')
plt.ylabel('Explained Variance Ratio')
plt.xticks(np.arange(1, len(ev) + 1, 1))
plt.grid()
plt.show()

# Optionally, you can also plot the cumulative explained variance
cumulative_variance = np.cumsum(ev)
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--')
plt.title('Cumulative Explained Variance Plot')
plt.xlabel('n_components')
plt.ylabel('Cumulative Explained Variance')
plt.xticks(np.arange(1, len(cumulative_variance) + 1, 1))
plt.grid()
plt.axhline(y=0.90, color='r', linestyle='--')  # Example threshold at 90%
plt.show()

Our PCA will perform best when using 3 components, as seen by the plots above. SVD Could have also been used to determine the number of components, as we have previously done in class.

In [ ]:
pca = PCA(n_components= 3)
pca_df = pd.DataFrame(pca.fit_transform(X_train))
pca_df['Labels'] = nbadf_2.PPG_Bins

plt.figure()
plt.scatter(pca_df[0], pca_df[1],
            c=pca_df['Labels'],
            alpha=0.6)
plt.title('PCA Clusters')
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.colorbar(label='Cluster Label')
plt.grid()
plt.show()

In [ ]:
feature = 0
hi_feat = -1
hi_weight = -1
for i in pca.components_[0]:
    if i > hi_weight and i > 0:
        hi_feat = feature
        hi_weight = i
    feature += 1

print('Feature with the heighest positive weight:')
print(hi_feat, hi_weight)

Our dataset results in poor PCA clustering for a few reasons.
First, our PCA graph has a wedge shape due to the abscence of negative values. This can also be seen by plotting a histogram of PPG; our data is right skewed as more players have small points scored per game, while a few "Power Forwards" score the bulk of a team's points. The second reason is most likely due to the standardization performed on each variable prior to PCA.
As such, we will be using SVD with 3 components

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=3, n_iter=1000, random_state=76)
svd.fit(X_train)
sv = svd.singular_values_

right_matrix = pd.DataFrame(svd.components_)
right_matrix.shape # lets check the shape

left_matrix = pd.DataFrame(svd.fit_transform(X_train))/ sv
left_matrix.shape

To better visualize what our SVD looks like, we will use a pairplot showing each of the 3 components of the left and right matrices we have created.

In [ ]:
import seaborn as sns
def generate_plot(svd):
    sv = svd.singular_values_
    left_matrix = pd.DataFrame(svd.transform(X_train)) / sv
    right_matrix = pd.DataFrame(svd.components_)
    
    # Left matrix pairplot
    plt.figure(figsize=(10, 8))
    sns.pairplot(left_matrix)
    plt.suptitle("Pairplot of Left Matrix (Transformed Features)", y=1.02)
    plt.show()
    
    # Right matrix pairplot
    plt.figure(figsize=(10, 8))
    sns.pairplot(right_matrix.T)
    plt.suptitle("Pairplot of Right Matrix (Components)", y=1.02)
    plt.show()

In [ ]:
generate_plot(svd)

Now we can see why our PCA did not produce good clusters. Our vertical plots of n=2 and n=3 show a unmimodal distribution. Had our data shown multiple peaks, clustering would have become more noticeable.

Due to the small amount of observations in our right matrix, very little significant information is available that the left matrix is does not already produce.

Finally, we can perform logistic regression by fitting a regressor on our transformed training data, predicting our test data, and comparing our resulting predictions with our actual y values using classification_report

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Step 1: Transform both training and testing data using SVD
X_train_svd = pd.DataFrame(svd.transform(X_train), index=X_train.index)
X_test_svd = pd.DataFrame(svd.transform(X_test), index=X_test.index)

# Step 2: Fit a classifier on the transformed training data
classifier = LogisticRegression()
classifier.fit(X_train_svd, y_train)

# Step 3: Predict on the transformed test data
y_pred = classifier.predict(X_test_svd)

# Optional: Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Our test accuracy hovers around roughly 84%. We would need to run these models again without seeding to determine its true efficacy

In [ ]:
y_pred2 = classifier.predict(X_train_svd)

# Optional: Evaluate the model
print("Accuracy:", accuracy_score(y_train, y_pred2))
print("\nClassification Report:\n", classification_report(y_train, y_pred2))

Running the prediction on our training data also produces a similar accuracy score. This is indicative of the SVD improving our predictive probability when classifying Points Per Game as a categorical variable. Should we increase the bins (for example, from quartiles to deciles), our accuracy would decrease slightly, but it would provide a significantly better predictor in comparison to running linear regression on the normalized PPG.